# Text Generation with Neural Networks

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

## The Data
Shakespeare's works 


In [4]:
path_to_file = 'shakespeare.txt'

In [5]:
text = open(path_to_file, 'r').read()

In [6]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Text Processing

### Text Vectorization

In [8]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [11]:
# char_to_ind

In [10]:
ind_to_char = np.array(vocab)

In [15]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [16]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [17]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

We now have a mapping we can use to go back and forth from characters to numerics.

In [20]:
sample = text[180:200]
sample

't bear his memory:\n '

In [21]:
encoded_text[180:200]

array([75,  1, 57, 60, 56, 73,  1, 63, 64, 74,  1, 68, 60, 68, 70, 73, 80,
       21,  0,  1])

## Creating Batches

In [22]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [23]:
line = "From fairest creatures we desire increase"

In [24]:
len(line)

41

In [25]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [26]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [27]:
seq_len = 120

In [28]:
total_num_seq = len(text)//(seq_len+1)

In [29]:
total_num_seq

45005

In [30]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [31]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [32]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [33]:
dataset = sequences.map(create_seq_targets)

In [34]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [35]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [36]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

##  Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. 

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [38]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [40]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [41]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)



In [42]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [43]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [44]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


##  Training the model


In [46]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [47]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 1.99773000e-03, -4.05380921e-03,  6.34850562e-03, ...,
          5.99220209e-03, -2.67351815e-03, -1.18645444e-03],
        [-6.28103968e-04, -3.78927402e-03, -1.23591442e-03, ...,
          3.86248529e-03, -2.77028861e-03, -2.42896564e-03],
        [-2.11126218e-03, -3.32806259e-03,  6.21052692e-04, ...,
          1.14502596e-04, -1.43635750e-03, -2.51611206e-03],
        ...,
        [-4.08423645e-03,  5.76018775e-03,  4.88405488e-03, ...,
         -8.08194093e-03, -3.07812635e-03,  4.72540967e-03],
        [-2.86188233e-03,  1.02238264e-02,  5.78609528e-03, ...,
         -1.02221258e-02, -4.24811989e-03,  7.89846294e-03],
        [-1.43991131e-03,  1.25518609e-02,  5.80580346e-03, ...,
         -1.18512353e-02, -5.35408175e-03,  9.92190093e-03]],

       [[ 4.70541604e-03, -1.42527872e-03, -4.06456273e-03, ...,
          8.41595605e-03, -1.80663401e-03,  4.40764241e-04],
        [ 1.00740558e-03,  6.63471734e-03,  4.4

In [48]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [49]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[36],
       [26],
       [73],
       [50],
       [78],
       [28],
       [58],
       [19],
       [83],
       [48],
       [ 2],
       [80],
       [14],
       [28],
       [65],
       [11],
       [61],
       [44],
       [71],
       [41],
       [81],
       [ 0],
       [15],
       [44],
       [47],
       [59],
       [53],
       [10],
       [23],
       [ 2],
       [ 0],
       [20],
       [80],
       [ 9],
       [ 4],
       [29],
       [18],
       [ 5],
       [22],
       [52],
       [73],
       [26],
       [27],
       [ 6],
       [31],
       [30],
       [59],
       [66],
       [41],
       [83],
       [12],
       [65],
       [39],
       [51],
       [68],
       [14],
       [33],
       [66],
       [51],
       [56],
       [31],
       [22],
       [63],
       [ 7],
       [52],
       [26],
       [78],
       [42],
       [11],
       [ 4],
       [17],
       [51],
       [55],
   

In [50]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [51]:
sampled_indices

array([36, 26, 73, 50, 78, 28, 58, 19, 83, 48,  2, 80, 14, 28, 65, 11, 61,
       44, 71, 41, 81,  0, 15, 44, 47, 59, 53, 10, 23,  2,  0, 20, 80,  9,
        4, 29, 18,  5, 22, 52, 73, 26, 27,  6, 31, 30, 59, 66, 41, 83, 12,
       65, 39, 51, 68, 14, 33, 66, 51, 56, 31, 22, 63,  7, 52, 26, 78, 42,
       11,  4, 17, 51, 55,  7, 14,  5,  9, 78, 47, 50, 32, 68, 48, 81, 62,
       40,  7, 79, 80, 16, 43, 42, 45, 21, 61, 51, 63, 34, 32, 43, 47,  3,
        0, 24, 68, 10, 15, 38, 66, 41,  9, 79, 41, 83, 71, 60, 23, 64, 65,
        0], dtype=int64)

In [52]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

he
    May spend our wonder too, or take off thine
    By wond'ring how thou took'st it.
  LAFEU. Nay, I'll fit you,
   


Next Char Predictions: 

KArYwCc8}W!y3Cj0fSpPz
4SVd].<!
9y-&D7';[rAB(FEdkP}1jNZm3HkZaF;h)[AwQ0&6Z`)3'-wVYGmWzgO)xy5RQT:fZhIGRV"
>m.4MkP-xP}pe<ij



After confirming the dimensions are working, let's train our network!

In [53]:
epochs = 30

In [55]:
model.fit(dataset,epochs=epochs)

##  Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [0]:
model.save('shakespeare_gen.h5') 

In [56]:
from tensorflow.keras.models import load_model

In [57]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [59]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [60]:
print(generate_text(model,"flower",gen_size=1000))

flower
  F is touch'd in this ince?
    The Duke will not perh people: know they are not seeates out:
    Or speak with me, and we will persever
wars.
  ANIEL. Mostly chones, lick money, put indeed her rumorable heir
    One spirit, who, if her beauty 'tis your tour;
    Train of the books, I would not laid to Fredce;
    But tell us twice as womanour, if the Tribunes choose Claudio;
    But know art thou the Sebastian?
  SECOND LORD. Nay, not I.
  Pol. Marry, hard for my sword mine eye my mind, I would
    Make Hector all; I will encounter surt.
  YORK. He hath wrong to be dishonoured,
    He makes fur'd his sister of his aid
    To breathe our premedimages creature.  
  CLOWN. Yet what, ay' honors in the fear?
  TALBOT. O, then, entreat him there!' It is as half,
    Ah, which with other lord? O by his powers;
    And in a thousand in thy heart with files,
    And in the time for knowing with Imodes
    Without them there to be disguis'd;
    His help, heard more, join where my worth

### Done!
